# 04-01 - Vector Store - Text Search

## Overview 


## Environment

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.InMemory, 1.38.0-preview"
#r "nuget: DotNetEnv, 3.1.1"

using System.IO;
using System.ClientModel;
using System.ComponentModel;
using DotNetEnv;
using Microsoft.Extensions.VectorData;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Connectors.InMemory;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

string apiKey = Env.GetString("SK_OPENAI_APIKEY");
string endpoint = Env.GetString("SK_OPENAI_ENDPOINT");
string embeddingDeployment = Env.GetString("SK_OPENAI_EMBEDDING_DEPLOYMENT");

Console.WriteLine($"Configuration loaded...");

## Kernel


In [ ]:
#pragma warning disable SKEXP0001, SKEXP0010 

Kernel kernel = Kernel.CreateBuilder()
    .AddAzureOpenAITextEmbeddingGeneration(
        apiKey: apiKey, 
        endpoint: endpoint, 
        deploymentName: embeddingDeployment
    )
.Build();

Console.WriteLine($"Kernel created...");

## Define Storage Model

In [ ]:
public sealed class DataStorageModel
{
    #pragma warning disable SKEXP0001
    [VectorStoreRecordKey]
    [TextSearchResultName]
    public Guid Key { get; init; }

    [VectorStoreRecordData]
    [TextSearchResultValue]
    public string Text { get; init; } = "Empty Record - No Data"; 

    [VectorStoreRecordVector(3072)]
    public ReadOnlyMemory<float> Embedding { get; init; }
    
    [TextSearchResultLink]
    public string Link {get; set;} = "";
    #pragma warning restore SKEXP0001
}

Console.WriteLine($"DataStorageModel defined...");

## Create Vector Store

In [ ]:
#pragma warning disable SKEXP0001, SKEXP0010 

ITextEmbeddingGenerationService textEmbeddingGenerationService = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

string[] textData = new string[]
{
    "The Super Sports Championship is an exhilarating physics competition where developers answer multiple questions, earning 10 points for each correct answer, with the first team to reach 50 points emerging victorious.",
    "The Super Bowl is the annual championship game of the National Football League (NFL) and one of the biggest sporting events in the United States.",
    "Semantic Kernel is a lightweight, open-source development kit that allows developers to easily integrate the latest AI models into their C#, Python, or Java codebase."
};

IList<ReadOnlyMemory<float>> vectorData = 
    await textEmbeddingGenerationService.GenerateEmbeddingsAsync(textData);

InMemoryVectorStore inMemoryStore = new InMemoryVectorStore();
string collectionName = "SportEventsKnowHow";

IVectorStoreRecordCollection<Guid, DataStorageModel> recordCollection = 
    inMemoryStore.GetCollection<Guid, DataStorageModel>(collectionName);
await recordCollection.CreateCollectionIfNotExistsAsync();

List<DataStorageModel> dataStorageRecords = textData
    .Select((text, index) => 
    {
        var key = Guid.NewGuid();
        return new DataStorageModel
        {
            Key = key,
            Text = text,
            Embedding = vectorData[index],
            Link = $"data:citation/text;{key}"
        };
    })
    .ToList();

Console.WriteLine($"DataStorageRecords created: ");
await foreach(Guid recordGuid in recordCollection.UpsertBatchAsync(dataStorageRecords))
{
    Console.WriteLine($"\t{recordGuid}");
} 

## Query Vector Store

In [ ]:
#pragma warning disable SKEXP0001, SKEXP0010
VectorStoreTextSearch<DataStorageModel> textSearch = 
    new VectorStoreTextSearch<DataStorageModel>(recordCollection, textEmbeddingGenerationService);


string query = "What is the Super Sports Championship";
// simple string as search result
KernelSearchResults<string> simpleResponse = await textSearch.SearchAsync(query, new() { Top = 1, Skip = 0 });
await foreach (var result in simpleResponse.Results)
{
    Console.WriteLine($"SearchAsync(): {result}");
}


// text search result as search result
KernelSearchResults<TextSearchResult> textSearchResults = await textSearch.GetTextSearchResultsAsync(query, new() { Top = 1, Skip = 0 });
await foreach (var result in textSearchResults.Results)
{
    Console.WriteLine($"GetTextSearchResultsAsync(): Link: {result.Link}, Name: {result.Name}, Value: {result.Value}");
}

// full data storage model as search result
KernelSearchResults<object> fullResults = await textSearch.GetSearchResultsAsync(query, new() { Top = 1, Skip = 0 });
await foreach (var result in fullResults.Results)
{
    DataStorageModel dataStorageModelResponse = result as DataStorageModel ?? new DataStorageModel(); 
    Console.WriteLine($"GetSearchResultsAsync(): Key: {dataStorageModelResponse.Key}, Text: {dataStorageModelResponse.Text}, Link: {dataStorageModelResponse.Link}");
}
